<a href="https://colab.research.google.com/github/orlandoalexander/Moda-Personal-Assistant/blob/playground/ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Input
import pandas as pd
import numpy as np


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 32.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [9]:
!pip install git+https://github.com/orlandoalexander/Moda-Personal-Assistant.git@preproc_package


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/orlandoalexander/Moda-Personal-Assistant.git (to revision preproc_package) to /tmp/pip-req-build-y42nfbje
  Running command git clone --filter=blob:none --quiet https://github.com/orlandoalexander/Moda-Personal-Assistant.git /tmp/pip-req-build-y42nfbje
  Running command git checkout -b preproc_package --track origin/preproc_package
  Switched to a new branch 'preproc_package'
  Branch 'preproc_package' set up to track remote branch 'preproc_package' from 'origin'.
  Resolved https://github.com/orlandoalexander/Moda-Personal-Assistant.git to commit d1345bf7f54ff84c6a3297856b65cf476837a801
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.8 MB/s eta 0:0

In [5]:
# !pip install -r requirements.txt

In [20]:
!unzip '/content/drive/MyDrive/attribute_img/img.zip' -d '/content/Moda-Personal-Assistant'


Streaming output truncated to the last 5000 lines.
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000019.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000020.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000021.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000022.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000023.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000024.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000025.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000026.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000027.jpg  
  inflating: /content/Moda-Personal-Assistant/img/Chiffon_Button-Front_Top/img_00000028.jpg  
  inflati

In [15]:
!pip install -U pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
!pip install Pillow==9.0.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preproc 0.0.0 requires Pillow==9.3.0, but you have pillow 9.0.0 which is incompatible.


In [17]:
from preproc.attributes import AttributePreproc


In [ ]:
!cat requirements.txt

In [ ]:

! ls

In [ ]:
!pwd

In [21]:
!unzip '/content/drive/MyDrive/attribute_img/img.zip'

Streaming output truncated to the last 5000 lines.
  inflating: img/Woven_Dolphin_Shorts/img_00000027.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000028.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000029.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000030.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000031.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000032.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000033.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000034.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000035.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000036.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000037.jpg  
   creating: img/Woven_Drawstring_Joggers/
  inflating: img/Woven_Drawstring_Joggers/img_00000001.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000002.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000003.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000004.jpg  
  inflating: img/Woven_Drawstring_J

In [18]:
prep = AttributePreproc('img', 
                        (224,224), 
                        'sleeves',
                        0.2) 
# e.g. AttributePreproc(folder/img, (256,256), 'fabric', 0.2)



In [24]:
!pip install -U pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.3.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.0.0
    Uninstalling Pillow-9.0.0:
      Successfully uninstalled Pillow-9.0.0


In [25]:
y_train, y_test, X_train, X_test  = prep.run()

Augmenting attribute 'long_sleeve'...


AttributeError: ignored

In [ ]:
from keras.applications import mobilenet


In [ ]:
X_train = mobilenet.preprocess_input(X_train)

In [ ]:
X_test = mobilenet.preprocess_input(X_test)

In [ ]:
import gc

In [ ]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from keras import Sequential
from keras.metrics import Precision, Recall
from keras.optimizers import Adam

from keras.applications import InceptionV3
from keras.applications import inception_v3
from keras.applications import ResNet50
from keras.applications import resnet
from keras.applications import MobileNetV2
from keras.applications import mobilenet_v2
from keras.applications import EfficientNetB0
from keras.applications import efficientnet

# attribute should be one of the following strings:
# 'design', 'sleeves', 'length', 'neckline', 'fabric', 'fit'

# model should be one of the following strings:
# 'inception', 'resnet', 'mobilenet', 'efficientnet'

class AttrModel:
    def __init__(self, attribute, epochs, model, input_shape, batch_size, final_layer_neurons,
                 X_train, X_test,
                 y_train, y_test, **kwargs):
        self.attribute = attribute
        self.model = model.lower()
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.final_layer_neurons = final_layer_neurons
        self.kwargs = kwargs
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            X_train, y_train, test_size=.5
        )
        self.X_test = X_test
        self.y_test = y_test
        del X_train, X_val, X_test, y_train, y_val, y_test
        gc.collect()
        self.cat_nums = {
            'design': 7,
            'sleeves': 3,
            'length': 2,
            'neckline': 4,
            'fabric': 6,
            'fit': 3
        }
        self.cat_num = self.cat_nums[self.attribute]
        self.activation = 'sigmoid' if self.attribute == 'length' else 'softmax'
        self.loss = 'binary_crossentropy' if self.attribute == 'length' else 'categorical_crossentropy'
        self.model = self.instantiate_model()  # calling the function below


    def instantiate_model(self):
        # import ipdb;
        # ipdb.set_trace()
        input = Input(self.input_shape)
        if self.model == 'inception':              # calling the chosen pretrained model
            base_model = InceptionV3(include_top=False, weights='imagenet',
                                    classes=self.cat_num, input_tensor=input)
        elif self.model == 'resnet':
            base_model = ResNet50(include_top=False, weights='imagenet',
                                  classes=self.cat_num, input_tensore=input)
        elif self.model == 'mobilenet':
            base_model = MobileNetV2(include_top=False, weights='imagenet',
                                     classes=self.cat_num, input_tensor=input)
        elif self.model == 'efficientnet':
            base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                        classes=self.cat_num, input_tensor=input)
        else:
            print('''No model found. Please pass one of the following:
                  inception, resnet, mobilenet, efficientnet''')

        base_model.trainable = False    # freeze layers

        pool = GlobalAveragePooling2D()
        dense = Dense(units=self.final_layer_neurons, activation='relu')
        prediction = Dense(units=self.cat_num, activation=self.activation)
        model = Sequential([base_model, pool, dense, prediction])
        print(self.loss) # check loss func
        model.compile(loss=self.loss, optimizer='adam',
                      metrics=['accuracy', Precision(), Recall()])

        return model

    def train(self):
        # import ipdb
        # ipdb.set_trace()
        self.model.history = self.model.fit(
            self.X_train, self.y_train,
            epochs=self.epochs, batch_size=self.batch_size,
            validation_split=0.2, verbose=1, 
            callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])
        return self.model.history

    def finetune(self,callbacks=[EarlyStopping(patience=3)],
                 loss='categorical_crossentropy',
                 metrics=['accuracy', Precision(), Recall()]):
        self.model.trainable = True     # unfreeze layers, then compile to save changes
        self.model.compile(
            optimizer=Adam(1e-5),                  # Very low learning rate
            loss=loss,
            metrics=metrics)
        self.model.fit(self.X_val, self.y_val,
                       epochs=self.epochs, batch_size=self.batch_size,
                       callbacks=[EarlyStopping(patience=2, monitor='val_accuracy', 
                                                restore_best_weights=True)],
                       validation_split=0.2)
        return self.model.history

    def evaluate(self):
        return self.model.evaluate(self.X_test, self.y_test, verbose=1)

    def predict(self, X):
        return self.model.predict(X)


In [ ]:
inception = AttrModel(attribute='neckline',
                      model='mobilenet',
                      input_shape=(224,224,3),
                      batch_size= 16,
                      epochs=50,
                      final_layer_neurons=100,
                      X_train=X_train,
                      X_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

In [ ]:
inception.train()

In [ ]:
inception.finetune()

In [ ]:
inception.evaluate()

# New Section